In [4]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Concatenate

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"units": 16, "epochs": 10},
    {"units": 16, "epochs": 20},
    {"units": 32, "epochs": 10},
    {"units": 32, "epochs": 20},
]

# ─── Build dual-input classic RNN ───────────────────────────────────────────
def make_dual_rnn(timesteps, n_feats, n_opp, n_outputs, units):
    seq_in = Input(shape=(timesteps, n_feats), name="player_seq")
    x = SimpleRNN(units, name="rnn")(seq_in)
    opp_in = Input(shape=(n_opp,), name="opp_stats")
    m = Concatenate()([x, opp_in])
    h = Dense(16, activation="relu")(m)
    out = Dense(n_outputs)(h)
    model = Model([seq_in, opp_in], out)
    model.compile(optimizer="adam", loss="mse")
    return model

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8: continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single-fold evaluation ─────────────────────────────────────────────────
def eval_rnn_fold(fold_idx, hp):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])
    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # Scale sequence features
    ns, ts, nf = X_tr_seq.shape
    s_scaler = StandardScaler().fit(X_tr_seq.reshape(-1, nf))
    X_tr_seq = s_scaler.transform(X_tr_seq.reshape(-1, nf)).reshape(ns, ts, nf)
    X_te_seq = s_scaler.transform(X_te_seq.reshape(-1, nf)).reshape(-1, ts, nf)

    # Scale opponent features
    o_scaler = StandardScaler().fit(X_tr_opp)
    X_tr_opp = o_scaler.transform(X_tr_opp)
    X_te_opp = o_scaler.transform(X_te_opp)

    # Train RNN
    model = make_dual_rnn(ts, nf, len(OPP_FEATS), len(TARGET_COLS), hp["units"])
    model.fit([X_tr_seq, X_tr_opp], y_tr, epochs=hp["epochs"], batch_size=32, verbose=0)

    # Predict
    preds = model.predict([X_te_seq, X_te_opp], verbose=0)
    # Compute per-target metrics
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    # evaluate each fold
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_rnn_fold)(fold, hp) for fold in range(1, 11)
    )
    # collect into DataFrame
    df = pd.DataFrame(fold_metrics)
    df["fold"] = np.arange(1, len(df)+1)
    df["units"]  = hp["units"]
    df["epochs"] = hp["epochs"]
    all_results.append(df)

# concatenate all hyperparameter runs
final_df = pd.concat(all_results, ignore_index=True)
# reorder columns
cols = ["units","epochs","fold"] + [c for c in final_df.columns if c not in ("units","epochs","fold")]
final_df = final_df[cols]
# display
final_df


2025-05-04 20:51:21.461917: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 20:51:21.461944: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-04 20:51:21.461956: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-04 20:51:21.461973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-04 20:51:21.461981: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-05-04 20:51:21.480855: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 20:51:21.480877: I metal_plugin/src/device/metal_device.cc:296] syste

units  epochs  fold   PTS_rmse    PTS_mae    PTS_r2  REB_rmse   REB_mae  \
0      16      10     1  16.787566  14.312174 -2.701163  6.298770  5.194901   
1      16      10     2   7.753962   6.181445  0.212162  3.558493  2.778291   
2      16      10     3   6.650854   5.310303  0.384845  3.274387  2.445627   
3      16      10     4   7.145864   5.600884  0.359548  3.431015  2.590054   
4      16      10     5   6.788057   5.303527  0.405860  3.022318  2.297702   
5      16      10     6   7.030973   5.498415  0.393294  2.891065  2.271684   
6      16      10     7   6.994057   5.622680  0.355034  2.953778  2.251735   
7      16      10     8   7.174744   5.481125  0.376106  2.940495  2.254074   
8      16      10     9   6.877628   5.414966  0.325174  3.000644  2.356623   
9      16      10    10   7.309536   5.737529  0.337434  2.950461  2.294749   
10     16      20     1  16.413401  13.953371 -2.538016  5.991222  4.892034   
11     16      20     2   6.911594   5.480947  0.374041  3.122874  2.439201   
12     16      20     3   6.695502   5.404640  0.376559  3.094622  2.339018   
13     16      20     4   7.017767   5.536269  0.382304  2.863058  2.217233   
14     16      20     5   6.810850   5.324644  0.401863  2.870863  2.203249   
15     16      20     6   7.091421   5.551029  0.382817  2.842745  2.202224   
16     16      20     7   6.969378   5.552477  0.359578  3.004765  2.277554   
17     16      20     8   7.115869   5.421533  0.386304  2.877188  2.202897   
18     16      20     9   6.784407   5.281757  0.343344  2.768581  2.121522   
19     16      20    10   7.299269   5.724594  0.339294  2.883346  2.233336   
20     32      10     1  17.154084  14.687842 -2.864539  5.989685  4.906094   
21     32      10     2   6.988089   5.538052  0.360108  3.155855  2.465255   
22     32      10     3   6.759170   5.407088  0.364645  3.073607  2.327167   
23     32      10     4   6.952433   5.492411  0.393752  3.164054  2.424582   
24     32      10     5   7.027395   5.444617  0.363224  3.361871  2.498683   
25     32      10     6   7.161334   5.582985  0.370588  3.136535  2.418449   
26     32      10     7   6.973879   5.496412  0.358751  2.926257  2.242601   
27     32      10     8   7.144624   5.432026  0.381334  2.849674  2.225446   
28     32      10     9   6.806024   5.367889  0.339152  2.795636  2.133412   
29     32      10    10   7.283925   5.732111  0.342069  2.935278  2.245006   
30     32      20     1  16.742814  14.242916 -2.681456  5.364653  4.259958   
31     32      20     2   6.891997   5.442933  0.377585  3.214624  2.532895   
32     32      20     3   6.663233   5.357082  0.382553  2.761978  2.114264   
33     32      20     4   7.044992   5.574214  0.377502  3.116481  2.353623   
34     32      20     5   6.821082   5.412192  0.400064  2.830505  2.166872   
35     32      20     6   7.096900   5.545060  0.381863  3.127238  2.419977   
36     32      20     7   7.031338   5.598099  0.348140  2.907228  2.211150   
37     32      20     8   7.190414   5.471796  0.373378  2.821933  2.184463   
38     32      20     9   6.740177   5.275417  0.351878  2.711348  2.079419   
39     32      20    10   7.284424   5.716896  0.341979  2.890505  2.202637   

      REB_r2  AST_rmse   AST_mae    AST_r2  
0  -2.275190  4.837455  3.616757 -1.325579  
1  -0.012132  2.571751  1.910366  0.213178  
2   0.158471  2.291787  1.763762  0.288864  
3   0.123935  2.402429  1.797648  0.257873  
4   0.335793  2.360596  1.801053  0.306484  
5   0.362363  2.394209  1.813745  0.275840  
6   0.334937  2.276086  1.761192  0.318743  
7   0.306822  2.507321  1.880256  0.247450  
8   0.200227  2.503695  1.880210  0.199613  
9   0.347087  2.503339  1.875933  0.270123  
10 -1.963165  4.478765  3.328446 -0.993489  
11  0.220504  2.395948  1.784373  0.317074  
12  0.248335  2.256039  1.739353  0.310876  
13  0.389970  2.290675  1.740661  0.325310  
14  0.400694  2.213063  1.693735  0.390462  
15  0.383499  2.243268  1.714412  0.364270  